In [55]:
import os
import openai
import sys
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
import yaml

In [58]:
# config.yml file contains OPENAI_API_KEY: "Your OPENAI API KEY"
with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)
OPENAI_API_KEY=config['OPENAI_API_KEY']

In [60]:
# Upload your pdf files to the folder called pdfs
pdf_folder_path = 'pdfs/'
loader = PyPDFDirectoryLoader(pdf_folder_path)
pages = loader.load()

In [61]:
print(pages[0])

page_content='Concrete scenario generation with\na focus on edge cases for the safety\nassessment of highly automated\nvehicles\nDhanoop Karunakaran\nA thesis submitted in fulfilment\nof the requirements of the degree of\nDoctor of Philosophy\nAustralian Centre for Field Robotics\nSchool of Aerospace, Mechanical and Mechatronic Engineering\nThe University of Sydney\nSeptember 2022' metadata={'source': 'pdfs/Thesis_modified.pdf', 'page': 0}


In [62]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=450,
    chunk_overlap=0, 
    separators=["\n\n", "\n", " ", ""]
)
splits = r_splitter.split_documents(pages)

In [63]:

embedding = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
persist_directory = 'docs/chroma/'
!rm -rf ./docs/chroma 
vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
)

In [64]:
print(vectordb._collection.count())

622


In [47]:
# Similarity search
question = "statistical validation"
docs = vectordb.similarity_search(question,k=3)
vectordb.persist()

In [48]:
for d in docs:
    print(d.metadata)

{'page': 34, 'source': 'pdfs/Thesis_modified.pdf'}
{'page': 36, 'source': 'pdfs/Thesis_modified.pdf'}
{'page': 96, 'source': 'pdfs/Thesis_modified.pdf'}


In [49]:
llm_name = "gpt-3.5-turbo"
llm = ChatOpenAI(model_name=llm_name, temperature=0,openai_api_key=OPENAI_API_KEY)
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever()
)
result = qa_chain({"query": question})

In [50]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)


In [51]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [52]:
question = "Is probability a class topic?"
result = qa_chain({"query": question})

Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-601xRLK4Z6lPYj59gNmPAwSP on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/account/billing to add a payment method..
Retrying langchain.embeddings.openai.embed_with_retry.<locals>._embed_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-embedding-ada-002 in organization org-601xRLK4Z6lPYj59gNmPAwSP on requests per min. Limit: 3 / min. Please try again in 20s. Contact us through our help center at help.openai.com if you continue to have issues. Please add a payment method to your account to increase your rate limit. Visit https://platform.openai.com/

In [53]:
print(result)

{'query': 'Is probability a class topic?', 'result': 'Probability is likely a class topic, as it is mentioned in the context in relation to calculating the probability of system satisfaction and finding the probability of success scenarios. Thanks for asking!', 'source_documents': [Document(page_content='phase, we validate with more challenging scenarios. Statistical characteristics of the\nsystem behaviour are computed by finding the probability of the system satisfying the\nspecifications. We calculate the probability by dividing the total number of success\nscenarios by the total sum of scenarios, as shown below:\npr=Successscenarios\nTotalscenarios(4.7)\nFailure scenarios with maximum reward are most likely high-risk cases. We argue', metadata={'page': 75, 'source': 'pdfs/Thesis_modified.pdf'}), Document(page_content='experiment based on the multivariate normal parameter space gives a significantly\nhigher likelihood of existing within the real-world data ( e−17), though still far 